In [ ]:
from keras import models
from keras import layers
import os
import cv2
import numpy as np
from keras.models import load_model
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
path = './opencv/create'

label_list = os.listdir(path)

Date_dir = "{:02d}{:02d}".format(datetime.today().month,datetime.today().day)

if os.path.exists(Date_dir) == False:
    os.mkdir(os.path.curdir + '/' +Date_dir)

weight_path = os.listdir(Date_dir)

model_save_path = Date_dir+'/'+str(len(weight_path))

os.mkdir(model_save_path)


In [ ]:
def load_src():
    source_data = []
    source_label = []
    for i in label_list:
        file_list = os.listdir(path+"/"+i)
        for j in file_list:
            temp = cv2.imread(path+"/"+i+"/"+j, cv2.IMREAD_GRAYSCALE)
            temp_cv = []
            for k in temp:
                temp_cv.extend(k)
            source_data.append(temp_cv)
            source_label.append(int(i))
    return source_data, source_label

In [ ]:
train_data = []
train_label = []


test_data = []
test_label = []

In [ ]:
max_train_count = []
label_list = os.listdir(path)
for i in label_list:
    file_list = os.listdir(path+"/"+i)
    max_train_count.append(int(len(file_list)))
max_train_count

In [ ]:
test_ind = [[0,300],
           [300,600],
           [600,900],
           [900,1200],
           [1200,1500]]

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
def train(batch, epoch, train_data, train_label, test_data, test_label,callback_list):
    network = models.Sequential()


    network.add(layers.Conv2D(128, kernel_size=(3,3), input_shape=(100,100,1), activation='relu'))
    network.add(layers.MaxPooling2D(pool_size=2))
    network.add(layers.Dropout(0.5))

    network.add(layers.Conv2D(64, (3,3), activation='relu'))
    network.add(layers.MaxPooling2D(pool_size=2))
    network.add(layers.Dropout(0.5))

    network.add(layers.Conv2D(32, (3,3), activation='relu'))
    network.add(layers.MaxPooling2D(pool_size=2))
    network.add(layers.Dropout(0.5))

    network.add(layers.Flatten())

    network.add(layers.Dropout(0.5))

    network.add(layers.Dense(10, activation='softmax'))
    
    network.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])
    
    hist = network.fit(train_data, train_label, epochs=epoch, batch_size=batch,
                   validation_data=(test_data,test_label), callbacks=callback_list)
    print("batch : {0} epoch : {1}".format(batch,epoch))
    return hist,network

In [ ]:
plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (14,4)
plt.figure(figsize=(8, 4))

In [ ]:
# batch size & epoch
batch = 64
epoch = 30

In [ ]:
for index in range(5):
    
    SPath = model_save_path+'/'+str(index)
    if os.path.exists(SPath)== False:
        os.mkdir(SPath)
    
    print("index : ",index)
    train_data = []
    train_label = []

    
    test_data = []
    test_label = []
    
    source_data, source_label = load_src()
    
    ind = 0
    for i in range(0,10):
        test_data.extend(source_data[ind+test_ind[index][0]:ind+test_ind[index][1]])
        test_label.extend(source_label[ind+test_ind[index][0]:ind+test_ind[index][1]])
        if(test_ind[index][0] != 0):
            train_data.extend(source_data[ind:ind+test_ind[index][0]])
            train_label.extend(source_label[ind:ind+test_ind[index][0]])
        if(test_ind[index][1] != max_train_count[i]):
            train_data.extend(source_data[ind+test_ind[index][1]:ind+max_train_count[i]])
            train_label.extend(source_label[ind+test_ind[index][1]:ind+max_train_count[i]])
        ind += max_train_count[i]
                
    source_data = []
    source_label = []
    
    train_data = np.array(train_data)
    train_label = np.array(train_label)
    test_data = np.array(test_data)
    test_label = np.array(test_label)

    test_data = test_data.reshape((len(test_data),100,100,1))
    test_data = test_data.astype('float32')/255
    
    train_label = to_categorical(train_label,10)
    test_label = to_categorical(test_label,10)
    
    train_data = train_data.reshape((len(train_data),100,100,1))
    train_data = train_data.astype('float32')/255
    
    filepath= SPath+"/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

    callbacks_list = [checkpoint ]
    
    hist, network = train(batch, epoch, train_data, train_label,test_data, test_label, callbacks_list)
    
    
    plt.subplot(1, 2, 1)
    plt.plot(hist.history['loss'], 'b-', label="학습 성능")
    plt.plot(hist.history['val_loss'], 'r:', label="검증 성능")
    plt.title(str(index+1)+"way  훈련 중 비용함수 그래프")
    plt.ylabel("비용함수 값")
    plt.subplot(1, 2, 2)
    plt.title(str(index+1 )+"way  훈련 중 성능지표 그래프")
    plt.ylabel("성능지표 값")
    plt.plot(hist.history['acc'], 'b-', label="학습 성능")
    plt.plot(hist.history['val_acc'], 'r:', label="검증 성능")
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    test_loss, test_acc = network.evaluate(test_data,test_label)
    print("test_acc : ", test_acc)
    
    network.save(Date_dir+'/'+str(index+1)+'.h5')